In [ ]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from datetime import datetime, timedelta
import streamlit as st
from pypfopt import expected_returns, EfficientFrontier
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2014-01-01'
end_date = datetime.now() - timedelta(1)

# just grabbing the 'Adj Close' column
stock_data = utils.get_stock_data(tickers, start_date, end_date)

sp500_data = stock_data['^GSPC']
stock_data = stock_data.drop(['^GSPC'], axis=1)

stock_data.tail()

"""similar to portfolio/display.py, set the portfolio weights and calculate the performance
"""
risk_free_rate = 0.04
mu = expected_returns.mean_historical_return(stock_data)
S = utils.calculate_covariance_matrix(stock_data)

# Calculating the cumulative monthly returns of a portfolio of stocks with given weights:
min_risk, max_risk = utils.calculate_risk_extents(mu, S, risk_free_rate)
risk = (max_risk + min_risk) / 2

ef = EfficientFrontier(mu, S)
ef.efficient_risk(risk)
weights = ef.clean_weights()
weights = pd.Series(weights).reindex(stock_data.columns)
ef_returns, ef_volatility, ef_sharpe = ef.portfolio_performance(risk_free_rate)
print(f'ef weights:\n{weights}')
print(f'ef performance: {ef_returns, ef_volatility, ef_sharpe}')

# First, calculate the daily returns for each stock
daily_returns = stock_data.pct_change()
print(f'daily_returns tail:\n{daily_returns.tail()}')

# Calculate the weighted returns
weighted_returns = daily_returns.multiply(weights, axis=1)

# Then, calculate the portfolio return by summing across the rows (i.e., the weighted returns for each day)
portfolio_daily_returns = weighted_returns.sum(axis=1)
print(f'portfolio_returns tail:\n{portfolio_returns.tail()}')

# Finally, calculate the cumulative monthly returns
portfolio_cumulative_daily_returns = (1 + portfolio_daily_returns).cumprod() - 1
print(f'portfolio_returns tail:\n{portfolio_cumulative_daily_returns.tail()}')

# Resample the portfolio returns into monthly returns
portfolio_returns_monthly = portfolio_daily_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)

# Finally, calculate the cumulative monthly returns
portfolio_cumulative_monthly_returns = (1 + portfolio_returns_monthly).cumprod() - 1
print(f'portfolio_returns tail:\n{portfolio_cumulative_monthly_returns.tail()}')

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_daily_returns.index,
    y=portfolio_daily_returns,
    mode='lines',
    name='Weighted Portfolio Daily Returns'
))

fig.update_yaxes(title_text='Returns (%)', tickformat=".1%")
fig.update_layout(title='Weighted Portfolio Daily Returns')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_cumulative_daily_returns.index,
    y=portfolio_cumulative_daily_returns,
    mode='lines',
    name='Weighted Portfolio'
))

fig.update_yaxes(title_text='Returns (%)', tickformat=".1%")
fig.update_layout(title='Weighted Portfolio Cumulative Daily Returns')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_returns_monthly.index,
    y=portfolio_returns_monthly,
    mode='lines',
    name='Weighted Portfolio Monthly Returns'
))

fig.update_yaxes(title_text='Returns (%)', tickformat=".1%")
fig.update_layout(title='Weighted Portfolio Monthly Returns')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_cumulative_monthly_returns.index,
    y=portfolio_cumulative_monthly_returns,
    mode='lines',
    name='Weighted Portfolio'
))

fig.update_yaxes(title_text='Returns (%)', tickformat=".1%")
fig.update_layout(title='Weighted Portfolio Cumulative Monthly Returns')
fig.show()


In [ ]:
# rename first column to 'Returns'

print(f'portfolio_daily_returns head:\n{portfolio_daily_returns.head()}')
print(f'portfolio_cumulative_daily_returns head:\n{portfolio_cumulative_daily_returns.head()}')
print(f'portfolio_returns_monthly head:\n{portfolio_returns_monthly.head()}')
print(f'portfolio_cumulative_monthly_returns head:\n{portfolio_cumulative_monthly_returns.head()}')





In [ ]:

# TODO: whelp, the import is failing with Exception: Could not deserialize ATN with version <ctrl char> (expected 4).
# which upon search comes from antlr4 -> per pip list, antlr4-python3-runtime 4.12.0 is installed
# per https://github.com/antlr/antlr4/issues/3753, "Please regenerate lexer/parser using the new ANTLR 4.10.1", ???
# maybe need to follow the more detailed steps to fully install OpenBB Terminal so the SDK works?
# or... given it's in open source, maybe just clone the repo and run it as a local package?
from openbb_terminal.sdk import openbb

spy = openbb.stocks.load('SPY', start_date = '1990-01-01')


In [ ]:
from typing import Union

"""
https://docs.openbb.co/sdk/usage/intros/forecast

Tuple[List[TimeSeries], List[TimeSeries], List[TimeSeries], Optional[float], StatsForecast, Union[int, str]]

openbb.forecast.autoselect(data: Union[pd.Series, pd.DataFrame], 
                            target_column: str = "close", 
                            seasonal_periods: int = 7, 
                            n_predict: int = 5, 
                            start_window: float = 0.85, 
                            forecast_horizon: int = 5)

"""

model_table = Union[int, str]
adj_data, historical_forecast, predicted_forecast, precision, ets_model, model_table = openbb.forecast(portfolio_daily_returns)